### Imports necessary for running the program

In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun May  5 23:01:42 2019

@author: Ahmed
"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import precision_recall_fscore_support
import numpy as np
import pandas as pd
'''df = pd.read_excel("MLDataSet.xlsx")
df2 = pd.read_excel("ADRs.xlsx")
df3 = pd.read_excel("DS.xlsx")
df4 = pd.read_excel("Mental.xlsx")

df5=[df, df2,  df3, df4]

result = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'''
#result.to_excel('result.xlsx')


'df = pd.read_excel("MLDataSet.xlsx")\ndf2 = pd.read_excel("ADRs.xlsx")\ndf3 = pd.read_excel("DS.xlsx")\ndf4 = pd.read_excel("Mental.xlsx")\n\ndf5=[df, df2,  df3, df4]\n\nresult = pd.concat([df,df2,df3, df4], axis=1, join_axes=[df.index])'

In [2]:
def removeColumns(result):
    del result['Pain']
    del result['Content']
    del result['Filtered']
    del result['Stemmed']
    del result['big1']
    del result['big2']
    del result['small1']
    del result['small2']
    del result['Height']
    del result['Joined']
    del result['Posted']
    

In [3]:
def vectorizeCount(result):
    for i in range(len(result)):
        if result[i]==0:
            result[i]=0
        if result[i]>0 and result[i]<=3:
            result[i]=1
        if result[i]>3:
            result[i]=2
    return result
    '''if result.at[i,'MentalCount']==0:
        result.at[i,'MentalCount']=0
    if result.at[i,'MentalCount']>0 and result.at[i,'MentalCount']<=3:
        result.at[i,'MentalCount']=1
    if result.at[i,'MentalCount']>3:
        result.at[i,'MentalCount']=2


    if result.at[i,'DieaseCount']==0:
        result.at[i,'DieaseCount']=0
    if result.at[i,'DieaseCount']>0 and result.at[i,'DieaseCount']<=3:
        result.at[i,'DieaseCount']=1
    if result.at[i,'DieaseCount']>3:
        result.at[i,'DieaseCount']=2'''

### manipulating data to prepare for classification

In [63]:
def manipulate(result,labelfile,label):
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    result['Gender']=imp_mean.fit_transform(result['Gender'].values.reshape(-1, 1))
    for i in range(len(result)):
        if result.at[i,'Gender']=='Male':
            result.at[i,'Gender']=0
        if result.at[i,'Gender']=='Female':
            result.at[i,'Gender']=1
    result['Drug']=LabelEncoder().fit_transform(result['Drug'])
    if 'Unnamed: 0' in result:
        del result['Unnamed: 0'] 
    result['DrugFamily']=LabelEncoder().fit_transform(result['DrugFamily'])
    labels=labelfile.iloc[:][label]
    
    #removeColumns(result)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='median')
    result['Age']=imp_mean.fit_transform(result['Age'].values.reshape(-1, 1))
    if 'Height' in result:
        result['Height']=imp_mean.fit_transform(result['Height'].values.reshape(-1, 1))
    return result,labels

In [68]:

asklabel=pd.read_excel('asklabels.xlsx')
labels=asklabel.iloc[:]['MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME']

### Datasets

In [26]:
label='Hypertensive disease '
asklabel='Hypertensive disease '

#from collections import Counter
#X_res, y_res = sm.fit_resample(result, result_labels)
#print(Counter(result_labels))
#print(Counter(y_res))
def prepareDataset(label,asklab):
    from imblearn.over_sampling import SMOTENC
    
    sm = SMOTENC(random_state=42, categorical_features=[0, 1,3])
    
    result=pd.read_excel('result_reduced.xlsx')
    weighted=pd.read_excel('Weighted_reduced.xlsx')
    blood=pd.read_excel('blood_reduced.xlsx')
    ask=pd.read_excel('askapatient_reduced.xlsx')

    resultlabel=pd.read_excel('resultlabels.xlsx')
    weightedlabel=pd.read_excel('Weightedlabels.xlsx')
    bloodlabel=pd.read_excel('bloodlabels.xlsx')
    asklabel=pd.read_excel('asklabels.xlsx')

    result,result_labels=manipulate(result,resultlabel,label)
    weighted,weighted_labels=manipulate(weighted,weightedlabel,label)
    blood,blood_labels=manipulate(blood,bloodlabel,label)
    ask,ask_labels=manipulate(ask,asklabel,asklab)
    if 'Count' in label:
        result_labels = vectorizeCount(result_labels)
        weighted_labels = vectorizeCount(weighted_labels)
        blood_labels=vectorizeCount(blood_labels)
    if 'Count' in asklab:
        ask_labels = vectorizeCount(ask_labels)
    del result['weights2']
    del result['Height']
    
    
    #Applying SMOTENC
    result,result_labels=sm.fit_resample(result,result_labels)
    weighted,weighted_labels=sm.fit_resample(weighted,weighted_labels)
    blood,blood_labels=sm.fit_resample(blood,blood_labels)
    ask,ask_labels=sm.fit_resample(ask,ask_labels)
    return result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels


In [6]:
'''for i in weighted:
    if i not in result:
        if i != 'weights2':
            del weighted[i]
weighted.to_excel('Weighted.xlsx')'''

"for i in weighted:\n    if i not in result:\n        if i != 'weights2':\n            del weighted[i]\nweighted.to_excel('Weighted.xlsx')"

In [7]:
#ask_labels

In [8]:
'''for i in weighted:
    print (weighted[i])'''

'for i in weighted:\n    print (weighted[i])'

In [9]:
def get_score(model, X_train,X_test,Y_train,Y_test):
    model.fit(X_train,Y_train)
    return model.score(X_test,Y_test)

### 10-Fold Cross validation

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
#cross_val_score(RandomForestClassifier(n_estimators=1000),result,result_labels,cv=10)

In [11]:
#This was used for some other no longer existing purposes
def getMissingPercentage(feature):
    #nancount = int(result[result[feature].isnull()][feature].shape[0])
    nancount = int(result[result[feature]==1][feature].shape[0])
    size=int(result.shape[0])
    print (nancount)
    print ((nancount*100)/size)
#getMissingPercentage('Pvc')

In [12]:
'''from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
kf = KFold(n_splits=10)

scores_rf=[]

for train_index, test_index in kf.split(result):
    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]
    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))'''

'from sklearn.model_selection import KFold\nfrom sklearn.ensemble import RandomForestClassifier\nkf = KFold(n_splits=10)\n\nscores_rf=[]\n\nfor train_index, test_index in kf.split(result):\n    X_train, X_test, Y_train, Y_test = result.iloc[train_index], result.iloc[test_index], result_labels.iloc[train_index],result_labels.iloc[test_index]\n    scores_rf.append(get_score(RandomForestClassifier(n_estimators=400),X_train, X_test, Y_train, Y_test))'

### Random Forest Classifier Function 

In [41]:
def randomForest(result,result_labels,estimators,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    
    X_train, X_test, Y_train, Y_test = train_test_split(result, result_labels, test_size=0.30, random_state=42)
    print(Counter(result_labels))
    clf = RandomForestClassifier(n_estimators=estimators, max_depth=2,random_state=0)
    clf.fit(X_train, Y_train)
    pred=clf.predict(X_test)
    accuracy=(accuracy_score(Y_test, pred))
    #print (accuracy)
    scores_rf=(precision_recall_fscore_support(Y_test,pred, average='macro'))
    #print(DataSet)
    #print (pred)
    #print(classification_report(Y_test,pred,labels=[0,1,2]))
        
    return accuracy,scores_rf

### Random Forest

In [70]:
label='MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME '
asklabel='MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME'
res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,2000,100):
    print (estimators)
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
    res_acc,res_scores=randomForest(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest(ask,ask_labels,estimators,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
res_Stats.to_excel('Forrestresult_Stats'+label+'.xlsx') 
wei_Stats.to_excel('Forrestweight_Stats'+label+'.xlsx') 
bld_Stats.to_excel('Forrestblood_Stats'+label+'.xlsx') 
ask_Stats.to_excel('Forrestask_Stats'+asklabel+'.xlsx') 

100
Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.5490463215258855 # Scores: (0.5520980261150017, 0.5469019188498783, 0.5364490617517148, None)
Wei: Accuracy: 0.559322033898305 # Scores: (0.5616359447004609, 0.5619212962962963, 0.5591954022988506, None)
Bld: Accuracy: 0.6794258373205742 # Scores: (0.6808726067746687, 0.6799780179520059, 0.679161415969756, None)
Ask: Accuracy: 0.6836734693877551 # Scores: (0.6856654841676522, 0.6839255499153976, 0.6830050867353594, None)
200
Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.5599455040871935 # Scores: (0.5616141593475041, 0.5583971959840789, 0.5533791102696695, None)
Wei: Accuracy: 0.559322033898305 # Scores: (0.5616359447004609, 0.5619212962962963, 0.5591954022988506, None)
Bld: Accuracy: 0.6889952153110048 # Scores: (0.6912280701754385, 0.6896867558160835, 0.6885388724579866, None)
Ask: Accur

Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.553133514986376 # Scores: (0.5567272727272727, 0.5509713063625022, 0.5404373587441207, None)
Wei: Accuracy: 0.559322033898305 # Scores: (0.5616359447004609, 0.5619212962962963, 0.5591954022988506, None)
Bld: Accuracy: 0.6842105263157895 # Scores: (0.6860250737463127, 0.6848323868840447, 0.6838558855885589, None)
Ask: Accuracy: 0.6785714285714286 # Scores: (0.6823838458885026, 0.6789275022777561, 0.6771511491097341, None)
1700
Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.5572207084468664 # Scores: (0.5609895392504088, 0.555114952771342, 0.5452682545582783, None)
Wei: Accuracy: 0.559322033898305 # Scores: (0.5616359447004609, 0.5619212962962963, 0.5591954022988506, None)
Bld: Accuracy: 0.6842105263157895 # Scores: (0.6860250737463127, 0.6848323868840447, 0.6838558855885589, None)
Ask: Accuracy:

### Random Forest Classifier Function 10-Fold

In [15]:
def average(scores):
    FScore=0
    Recall=0
    Precision=0

    for i in scores:
        Precision+=i[0]
        Recall+=i[1]
        FScore+=i[2]
    return (Precision/10,Recall/10,FScore/10)

In [22]:
def randomForest10Fold(result,result_labels,estimators,DataSet):
    from sklearn.model_selection import KFold
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import f1_score
    from collections import Counter
    kf = KFold(n_splits=10)

    accuracy=[]

    scores_rf=[]
    print(Counter(result_labels))
    count = 1
    for train_index, test_index in kf.split(result):
        X_train, X_test, Y_train, Y_test = result[train_index], result[test_index], result_labels[train_index],result_labels[test_index]
        clf = RandomForestClassifier(n_estimators=estimators, max_depth=2,random_state=0)
        clf.fit(X_train, Y_train)
        pred=clf.predict(X_test)
        accuracy.append(accuracy_score(Y_test, pred))
        scores_rf.append(precision_recall_fscore_support(Y_test,pred, average='micro'))
        
        #print(str(count)+DataSet)
        #print(f1_score(Y_test, pred, average='micro'))
        count+=1
        #print (pred)
        #print(classification_report(Y_test,pred,labels=[0,1]))
    accuracy = np.mean(accuracy,axis=0)   
    return accuracy,average(scores_rf)

### Random Forest with 10-Fold Cross Validation

In [69]:
label='MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME '
asklabel='MICROCEPHALY , EPILEPSY , AND DIABETES SYNDROME'

res_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
wei_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
bld_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
ask_Stats = pd.DataFrame(columns = ['Estimators' , 'Accuracy', 'Precision' , 'Recall','FScore'])
for estimators in range (100,2000,100):
    result,result_labels,weighted,weighted_labels,blood,blood_labels,ask,ask_labels=prepareDataset(label,asklabel)
    res_acc,res_scores=randomForest10Fold(result,result_labels,estimators,"Result")
    wei_acc,wei_scores=randomForest10Fold(weighted,weighted_labels,estimators,"Weighted")
    blood_acc,blood_scores=randomForest10Fold(blood,blood_labels,estimators,"Blood")
    ask_acc,ask_scores=randomForest10Fold(ask,ask_labels,1000,"Ask")
    print("Res: Accuracy: " + str(res_acc)+" # "+ "Scores: "+str(res_scores))
    print("Wei: Accuracy: " + str(wei_acc)+" # "+ "Scores: "+str(wei_scores))
    print("Bld: Accuracy: " + str(blood_acc)+" # "+ "Scores: "+str(blood_scores))
    print("Ask: Accuracy: " + str(ask_acc)+" # "+ "Scores: "+str(ask_scores))
    res_Stats=res_Stats.append({'Estimators':estimators,'Accuracy':res_acc,'Precision':res_scores[0],'Recall':res_scores[1],'FScore':res_scores[2]},ignore_index=True)
    wei_Stats=wei_Stats.append({'Estimators':estimators,'Accuracy':wei_acc,'Precision':wei_scores[0],'Recall':wei_scores[1],'FScore':wei_scores[2]},ignore_index=True)
    bld_Stats=bld_Stats.append({'Estimators':estimators,'Accuracy':blood_acc,'Precision':blood_scores[0],'Recall':blood_scores[1],'FScore':blood_scores[2]},ignore_index=True)
    ask_Stats=ask_Stats.append({'Estimators':estimators,'Accuracy':ask_acc,'Precision':ask_scores[0],'Recall':ask_scores[1],'FScore':ask_scores[2]},ignore_index=True)
res_Stats.to_excel('ForrestFoldresult_Stats'+label+'.xlsx') 
wei_Stats.to_excel('ForrestFoldweight_Stats'+label+'.xlsx') 
bld_Stats.to_excel('ForrestFoldblood_Stats'+label+'.xlsx') 
ask_Stats.to_excel('ForrestFoldask_Stats'+asklabel+'.xlsx') 


Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.26493308798929405 # Scores: (0.2649330879892941, 0.2649330879892941, 0.2649330879892941)
Wei: Accuracy: 0.6478947368421053 # Scores: (0.6478947368421053, 0.6478947368421053, 0.6478947368421053)
Bld: Accuracy: 0.4171635610766045 # Scores: (0.4171635610766045, 0.4171635610766045, 0.4171635610766045)
Ask: Accuracy: 0.6257604227833236 # Scores: (0.6257604227833237, 0.6257604227833237, 0.6257604227833237)
Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.27681331549013044 # Scores: (0.27681331549013044, 0.27681331549013044, 0.27681331549013044)
Wei: Accuracy: 0.6428947368421052 # Scores: (0.6428947368421052, 0.6428947368421052, 0.6428947368421053)
Bld: Accuracy: 0.4143478260869565 # Scores: (0.4143478260869565, 0.4143478260869565, 0.4143478260869565)
Ask: Accuracy: 0.6257604227833236 # Scores: (0.62576

Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.277638006022081 # Scores: (0.277638006022081, 0.277638006022081, 0.277638006022081)
Wei: Accuracy: 0.6589473684210526 # Scores: (0.6589473684210525, 0.6589473684210525, 0.6589473684210526)
Bld: Accuracy: 0.4214699792960662 # Scores: (0.4214699792960662, 0.4214699792960662, 0.4214699792960662)
Ask: Accuracy: 0.6257604227833236 # Scores: (0.6257604227833237, 0.6257604227833237, 0.6257604227833237)
Counter({0: 1223, 1: 1223})
Counter({0: 98, 1: 98})
Counter({0: 348, 1: 348})
Counter({0: 652, 1: 652})
Res: Accuracy: 0.2780461692873871 # Scores: (0.2780461692873871, 0.2780461692873871, 0.2780461692873871)
Wei: Accuracy: 0.6589473684210526 # Scores: (0.6589473684210525, 0.6589473684210525, 0.6589473684210526)
Bld: Accuracy: 0.4214699792960662 # Scores: (0.4214699792960662, 0.4214699792960662, 0.4214699792960662)
Ask: Accuracy: 0.6257604227833236 # Scores: (0.62576042278332

In [ ]:
#ask_scores[9]

In [ ]:
#result

In [ ]:
'''from collections import Counter
from imblearn.over_sampling import SMOTENC
sm = SMOTENC(random_state=42, categorical_features=[0, 1,3])
X_res, y_res = sm.fit_resample(result, result_labels)
print(Counter(result_labels))
print(Counter(y_res))
print(X_res)
print(y_res)
print(result_labels)'''

In [ ]:
average(wei_scores)

In [ ]:
print(result_labels.value_counts())
print(weighted_labels.value_counts())
print(blood_labels.value_counts())
print(ask_labels.value_counts())

In [ ]:
print(result_labels.value_counts())
print(weighted_labels.value_counts())
print(blood_labels.value_counts())
print(ask_labels.value_counts())

In [ ]:
res_scores[0]

In [ ]:
wei_acc


In [ ]:
ask_acc

In [ ]:
res_scores

In [ ]:
wei_scores

In [ ]:
ask_scores